# Intro

This notebook gives the design decision made to decide on the sizing of the dual port 8T SRAM cell

# Setup

## Imports

In [1]:
import os
from os.path import realpath
from typing import Optional, overload

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
import ipywidgets as widgets

import numpy as np
from textwrap import dedent
import matplotlib.pyplot as plt

from c4m.flexmem import SRAMCellDP8T
from c4m.pdk import sky130
prims = sky130.tech.primitives


## Setup PySpice

Compiled libraries for ngspice 36 are provided compiled for either CentOS 7 or Ubuntu20.04 so simulation can be performed without needing to install ngspice. One can comment out the setting of the `NGPSICE_LIBRARY_PATH` environment variable and then ngspice executable in the path will be used to find the shared library. When using own ngspice it is adviced to use version 35 or later to speed up the 
Also the provided `.spiceinit` file in the running directory is needed to get this speed up.

In [2]:
import os
from os.path import realpath
_pwd = realpath(os.curdir)
# _libngspice_path = f"{_pwd}/libngspice36_u2004.so"
# _libngspice_path = f"{_pwd}/libngspice36_centos7.so"
_libngspice_path = f"/home/verhaegs/software/mint20/stow/ngspice-36/lib/libngspice.so"
os.environ["NGSPICE_LIBRARY_PATH"] = str(_libngspice_path)

## Sky130 SRAM cell class

In [3]:
class SRAMCellDP8T_Sky130(SRAMCellDP8T):
    def __init__(self, *,
        name: str,
        pullup_mos=prims.pfet_01v8, pullup_l: float, pullup_w,
        pulldown_mos=prims.nfet_01v8, pulldown_l: float, pulldown_w: float,
        passgate_mos=prims.nfet_01v8, passgate_l: float, passgate_w: float,
    ):
        super().__init__(
            cktfab=sky130.cktfab, pyspicefab=sky130.pyspicefab, name=name,
            pullup_mos=pullup_mos, pullup_params={"l": pullup_l, "w": pullup_w},
            pulldown_mos=pulldown_mos, pulldown_params={"l": pulldown_l, "w": pulldown_w},
            passgate_mos=passgate_mos, passgate_params={"l": passgate_l, "w": passgate_w},
            nom_vdd=1.8, nom_corner="logic_ff", nom_temperature=25,
        )

# SRAM design

The classic 6 transistor single port SRAM cells consists of two back-to-back invertors (e.g a latch) with two nmos access transistors. The pmos of the invertors is called the pull-up transistor, the nmos the pull-down transistor and the access transistor are also called pass gates.

Designing an SRAM cell consists of determining l and w of the transistors. This will involve a trade-off between speed, area, data retention, power consumption and write-ability. Different test benches are defined for deriving metrics to allow to make the trade-off.

Several metrics are defined in order to allow to quantify the trade-off being made:
- Read Current (speed)
- Leakage Current (static power consumption)
- Static Noise Margin (non-destructive read)
- Write Trip Point (write-ability)
- Capacitive load on bit line (speed & dynamic power consumption)
- Capacitive load on word line (second order dynamic power consumption)

Speed is determined both by the read current and the capacitive load of the bitline; the read current says how fast it can decharge the bit line. The capacitive load on the bit line is determined by the w$_{PG}$ and of the parasitic capacitance of the  metal line for the bit line. The R of this bit line may also be contributing to the delay for large bit lines; e.g. with SRAM blocks with high number of rows.

# Manual exploration

First one can view manually the impact of the cell design on the different metrics.


In [21]:
@widgets.interact
def sram_metrics(
    corner=widgets.Dropdown(
        value="logic_sf", options=["logic_tt", "logic_ff", "logic_ss", "logic_sf", "logic_fs"],
        description="corner",
    ),
    temp=widgets.FloatText(value=85, description="temp. [℃]"),
    vdd=widgets.FloatText(value=1.62, description="Vdd [V]"),
    l_pu=widgets.FloatText(value=0.15, description="l_PU [µm]"),
    w_pu=widgets.FloatText(value=0.42, description="w_PU [µm]"),
    l_pd=widgets.FloatText(value=0.15, description="l_PD [µm]"),
    w_pd=widgets.FloatText(value=0.42, description="w_PD [µm]"),
    l_pg=widgets.FloatText(value=0.15, description="l_PG [µm]"),
    w_pg=widgets.FloatText(value=0.42, description="w_PG [µm]"),
    simulate=widgets.Checkbox(value=False, description="Simulate")
):
    if not simulate:
        print("Simulation disabled")
        return

    cell = SRAMCellDP8T_Sky130(
        name="sramsp6t",
        pullup_mos=prims.pfet_01v8, pullup_l=l_pu, pullup_w=w_pu,
        pulldown_mos=prims.nfet_01v8, pulldown_l=l_pd, pulldown_w=w_pd,
        passgate_mos=prims.nfet_01v8, passgate_l=l_pg, passgate_w=w_pg,
    )
    print("1/6")
    Iread = cell.Iread(corner=corner, vdd=vdd, temperature=temp)
    print("2/6")
    Ileak = cell.Ileak(corner=corner, vdd=vdd, temperature=temp)
    print("3/6")
    Pstatic = Ileak*vdd
    print("4/6")
    SNMhold = cell.SNM(read=False, corner=corner, vdd=vdd, temperature=temp)
    print("5/6")
    SNMread = cell.SNM(read=True, corner=corner, vdd=vdd, temperature=temp)
    print("6/6")
    WTP = cell.WTP(corner=corner, vdd=vdd, temperature=temp)
    printmd(dedent("""
        | metric | value |
        |-:|:-:|
        | I_read | {:.3f}mA |
        | I_leak | {:.3f}nA |
        | P_static | {:.3f}mW/Mbit |
        | SNM_hold | {:.1f}mV |
        | SNM_read | {:.1f}mV |
        | WTP | {:.1f}mV |
    """.format(
        Iread*1e3, Ileak*1e9, Pstatic*1e9,
        SNMhold*1e3, SNMread*1e3, WTP*1e3,
    )))

interactive(children=(Dropdown(description='corner', index=3, options=('logic_tt', 'logic_ff', 'logic_ss', 'lo…

# Worst case corners

In this chapter the worst case corner will be determined for each of the metrics.

Values considered:

* corners: logic_ss, logic_ff, logic_sf, logic_fs
* vdd: 1.62...1.98V
* temperature: -25...85°C


In [5]:
testcell = SRAMCellDP8T_Sky130(
    name="dp8t",
    pullup_mos=prims.pfet_01v8, pullup_l=0.15, pullup_w=0.42,
    pulldown_mos=prims.nfet_01v8, pulldown_l=0.15, pulldown_w=0.42,
    passgate_mos=prims.nfet_01v8, passgate_l=0.15, passgate_w=0.42,
)


## SNMread

In [20]:
temps = [-25, 25, 85]
corners = ["logic_tt", "logic_ss", "logic_ff", "logic_sf", "logic_fs"]

n_temps = len(temps)
n_corners = len(corners)

SNMread = np.zeros((n_corners, n_temps))
vdd = 1.62
s = f"___SNM_read [mV] (vdd={vdd:.2f}V)___\n\n"
s += "| | " + " | ".join([str(temp) for temp in temps]) + " |\n"
s += "|-:|" + n_temps*":-:|" + "\n"
for i_corner, corner in enumerate(corners):
    print(f"{i_corner + 1}/{len(corners)}")
    s += "| __" + corner + "__ |"
    for i_temp, temp in enumerate(temps):
        SNM = testcell.SNM(read=True, corner=corner, vdd=vdd, temperature=temp)
        SNMread[i_corner, i_temp] = SNM
        s+= " {:.1f} |".format(SNM*1e3)
    s += "\n"

printmd(s)

1/5
2/5
3/5
4/5
5/5


___SNM_read [mV] (vdd=1.62V)___

| | -25 | 25 | 85 |
|-:|:-:|:-:|:-:|
| __logic_tt__ | 562.7 | 549.2 | 531.9 |
| __logic_ss__ | 624.4 | 622.3 | 612.8 |
| __logic_ff__ | 451.3 | 431.2 | 412.4 |
| __logic_sf__ | 274.6 | 232.2 | 160.7 |
| __logic_fs__ | 734.7 | 727.7 | 715.8 |


## WTP

In [19]:
temps = [-25, 25, 85]
corners = ["logic_tt", "logic_ss", "logic_ff", "logic_sf", "logic_fs"]

n_temps = len(temps)
n_corners = len(corners)

WTP = np.zeros((n_corners, n_temps))
vdd = 1.62
s = f"___WTP [mV] (vdd={vdd:.2f}V)___\n\n"
s += "| | " + " | ".join([str(temp) for temp in temps]) + " |\n"
s += "|-:|" + n_temps*":-:|" + "\n"
for i_corner, corner in enumerate(corners):
    print(f"{i_corner + 1}/{len(corners)}")
    s += "| __" + corner + "__ |"
    for i_temp, temp in enumerate(temps):
        v = testcell.WTP(corner=corner, vdd=vdd, temperature=temp)
        WTP[i_corner, i_temp] = v
        s+= f" {v*1e3:.1f} |"
    s += "\n"

printmd(s)

1/5
2/5
3/5
4/5
5/5


___WTP [mV] (vdd=1.62V)___

| | -25 | 25 | 85 |
|-:|:-:|:-:|:-:|
| __logic_tt__ | 635.3 | 632.2 | 627.5 |
| __logic_ss__ | 564.4 | 566.7 | 564.1 |
| __logic_ff__ | 715.8 | 710.4 | 707.5 |
| __logic_sf__ | 878.2 | 892.8 | 940.9 |
| __logic_fs__ | 395.6 | 376.7 | 355.9 |


- Read current is mainly determined by the drive strength of the pass gate and the pull down. This is lowest for 'SS' process, lowest Vdd and highest temperature
- Highest leakage/static power consumption is seen for logic_ff process, highest Vdd and highest temperature
- SNM_read is lowest for the stronger nmos and weaker pmos (logic_sf corner), lowest Vdd and the highest temperature
- WTP is lowest for weaker nmos and stronger pmos (logic_fs corner), lowest Vdd; and higher temperature does not much influence on WTP

In [14]:
class SRAMCellDP8T_Sky130_WC(SRAMCellDP8T_Sky130):
    Vdd_nom = 1.8
    @property
    def Vdd_low(self):
        return 0.9*self.Vdd_nom
    @property
    def Vdd_high(self):
        return 1.1*self.Vdd_nom
    
    temp_nom = 25
    temp_low = -25
    temp_high = 85

    @property
    def Iread_corner(self):
        return {"vdd": self.Vdd_low, "corner": "logic_ss", "temperature": self.temp_high}
    def Iread(self, *, vwl1: Optional[float]=None, vwl2: Optional[float]=None, vpre: Optional[float]=None):
        return super().Iread(vwl1=vwl1, vwl2=vwl2, vpre=vpre, **self.Iread_corner)

    @property
    def Ileak_corner(self):
        return {"vdd": self.Vdd_high, "corner": "logic_ff", "temperature": self.temp_high}
    def Ileak(self, *, vpre: Optional[float]=None, floatbl: bool=False):
        return super().Ileak(vpre=vpre, floatbl=floatbl, **self.Ileak_corner)
    
    @property
    def SNM_corner(self):
        return {"vdd": self.Vdd_low, "corner": "logic_sf", "temperature": self.temp_high}
    @overload
    def SNM(self, *, read: bool=True, vwl1: None=None, vwl2: None=None, vpre: Optional[float]=None) -> float:
        ...
    @overload
    def SNM(self, *, read: None, vwl1: float, vwl2: Optional[float]=None, vpre: Optional[float]=None) -> float:
        ...
    def SNM(self, *,
        read: Optional[bool]=True, vwl1: Optional[float]=None, vwl2: Optional[float]=None,
        vpre: Optional[float]=None,
    ) -> float:
        return super().SNM(read=read, vwl1=vwl1, vwl2=vwl2, vpre=vpre, **self.SNM_corner)
    
    @property
    def WTP_corner(self):
        return {"vdd": self.Vdd_low, "corner": "logic_fs", "temperature": self.temp_high}
    def WTP(self, vwl1: Optional[float]=None, vwl2: Optional[float]=None, vpre: Optional[float]=None):
        return super().WTP(vwl1=vwl1, vwl2=vwl2, vpre=vpre, **self.WTP_corner)

In [18]:
@widgets.interact
def sram_metrics(
    l_pu=widgets.FloatText(value=0.15, description="l_PU [µm]"),
    w_pu=widgets.FloatText(value=0.42, description="w_PU [µm]"),
    l_pd=widgets.FloatText(value=0.15, description="l_PD [µm]"),
    w_pd=widgets.FloatText(value=0.42, description="w_PD [µm]"),
    l_pg=widgets.FloatText(value=0.15, description="l_PG [µm]"),
    w_pg=widgets.FloatText(value=0.42, description="w_PG [µm]"),
    simulate=widgets.Checkbox(value=False, description="Simulate")
):
    cell = SRAMCellDP8T_Sky130_WC(
        name="sramsp6t",
        pullup_l=l_pu, pullup_w=w_pu,
        pulldown_l=l_pd, pulldown_w=w_pd,
        passgate_l=l_pg, passgate_w=w_pg,
    )

    if not simulate:
        print("Simulation disabled")
        return

    print("1/6")
    Iread = cell.Iread()
    print("2/6")
    Ileak = cell.Ileak()
    print("3/6")
    Pstatic = Ileak*cell.Ileak_corner["vdd"]
    print("4/6")
    SNMhold = cell.SNM(read=False)
    print("5/6")
    SNMread = cell.SNM(read=True)
    print("6/6")
    WTP = cell.WTP()
    printmd(dedent(f"""
        | metric | value | corner | V_dd [V] | temp [℃] |
        |-:|:-:|:-:|:-:|:-:|
        | I_read | {Iread*1e6:.1f}µA | {cell.Iread_corner["corner"]} | {cell.Iread_corner["vdd"]:.2f} | {cell.Iread_corner["temperature"]:d} |
        | I_leak | {Ileak*1e9:.2f}nA | {cell.Ileak_corner["corner"]} | {cell.Ileak_corner["vdd"]:.2f} | {cell.Ileak_corner["temperature"]:d} |
        | P_static | {Pstatic*1e9:.1f}mW/Mbit | {cell.Ileak_corner["corner"]} | {cell.Ileak_corner["vdd"]:.2f} | {cell.Ileak_corner["temperature"]:d} |
        | SNM_hold | {SNMhold*1e3:.1f}mV | {cell.SNM_corner["corner"]} | {cell.SNM_corner["vdd"]:.2f} | {cell.SNM_corner["temperature"]:d} |
        | SNM_read | {SNMread*1e3:.1f}mV | {cell.SNM_corner["corner"]} | {cell.SNM_corner["vdd"]:.2f} | {cell.SNM_corner["temperature"]:d} |
        | WTP | {WTP*1e3:.1f}mV | {cell.WTP_corner["corner"]} | {cell.WTP_corner["vdd"]:.2f} | {cell.WTP_corner["temperature"]:d} |
    """))

interactive(children=(FloatText(value=0.15, description='l_PU [µm]'), FloatText(value=0.42, description='w_PU …

The minimum DRC compliant w of the transistor are relatively big for Sky130. This results in a stable and writeable cell with dimensions of all 6 transistor using minimum w of 0.42µm and l of 0.15µm. The SNM_read is lower than the single port cell though with 160mV. Increagin l of passgate to 0.16µm inceases SNM read to 218mV with minor change of read current from 34.2µA ro 33.9µA.
When drawing areaid.sc layer on top of nmos the w can be decreased to 0.36µm. When applying this to the pass gate and the pulldown transistor the l of the pass gate has to be increased to 0.17µm. This then results in a SNM read of 232.8mV with a reduction of the read current to 28.7µA.

Summary of candidate SRAM cell sizing:

| l_PU   | w_PU   | l_PD   | w_PD   | l_PG   | w_PG   | I_read | SNM_read | WTP   |
|--------|--------|--------|--------|--------|--------|--------|----------|-------|
| 0.15µm | 0.42µm | 0.15µm | 0.42µm | 0.15µm | 0.42µm | 34.2µA | 161mV    | 356mV |
| 0.15µm | 0.42µm | 0.15µm | 0.42µm | 0.16µm | 0.42µm | 33.9µA | 218mV    | 355mV |
| 0.15µm | 0.42µm | 0.15µm | 0.36µm | 0.17µm | 0.36µm | 28.7µA | 233mV    | 269mV |
